# TRANSFER LEARNING FOR CUSTOM OBJECT CLASSIFICATION (VGG16)

In this notebook we will explain step by step how to train a Custom Object Classifier, specifically a Sunflower Classifier that will learn to recognize the presence of sunflowers in new images.   
In order to achieve this goal, we will apply Transfer Learning techniques starting from a pre-trained VGG16 architecture and we will train our Custom Classifier on our own dataset (containing 50% images with sunflowers and 50% images without).   
Furthermore, the training involves GPUs (because they are much more efficient to train Neural Networks) and it is distributed across multiple GPUs.

## Importing libraries

We import Tensorflow, Keras, Pandas, Numpy, metrics from Sklearn, Matplotlib for visualization.

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf 
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model 
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, f1_score, auc
from numpy import argmax
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from numba import cuda

## GPUs check

First of all, let's check for available GPUs and choose the number of GPUs to use for distribute training.   


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

For istance, let's assume we have 2 GPUs and we want to use them all to perform our task.

In [ ]:
num_gpus = 2      # set number of GPUS to be used

## Loading train, validation and test CSV

We need to load 3 csv (train, validation and test csv) and each of them must include the following 2 columns:
- "Filenames" (image filename),
- "labels" (associated label)

In [ ]:
## LOADING TRAIN, VALIDATION AND TEST csv ##
############################################

path_csv = "/pasquale/CUSTOM_OBJECT_CLASSIFICATION_VGG16/"                 # loading path

df_train = pd.read_csv(path_csv + "train.csv", sep= ";")                   # read train.csv

df_validation = pd.read_csv(path_csv + "validation.csv", sep = ";")        # read validation.csv

df_test = pd.read_csv(path_csv + "test.csv", sep= ";")                     # read test.csv

### Pay attention: df_train, df_validation and df_test must be in this format !!!   
We can see that "labels" column in the dataframe shows:
- a list containing "sunflower" when the object is present in the image
- an empty list when the object is not present in the image
![title](csv_format_custom_object_classification.png)   


## Data Augmentation 
### (Rotating, Shifting, Zooming, Flipping)

Data Augmentation is a technique to increase the amount of relevant data in our dataset. This will allow us to train our neural network with additional synthetically modified data.     
Data augmentation is useful to improve performance and outcomes of deep learning models by forming new and different examples, starting from the original ones.   
We use ImageDataGenerator to perform real-time training data augmentation.

In [ ]:
## TRAINING DATA AUGMENTATION ##
################################

## Data Augmentation is performed only on training images

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
                    rescale = 1./255.,                   # rescaling factor
                    rotation_range = 40,                 # degree range for random rotations
                    width_shift_range = 0.2,             # range random width shift 
                    height_shift_range = 0.2,            # range random height shift
                    shear_range = 0.2,                   # shear intensity
                    zoom_range = 0.1,                    # range for random zoom
                    horizontal_flip = True,              # randomly flip inputs horizontally
                    fill_mode = "nearest"                # how to fill points outside the boundaries
                    )

# No augmentation in validation data
valid_datagen = ImageDataGenerator( rescale = 1./255. )

# No augmentation in test data
test_datagen = ImageDataGenerator( rescale = 1./255. )



# Flow training images from train dataframe in batches using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(
                                        
        dataframe = df_train,                      # dataframe with filenames and labels
        directory = '/pasquale/CUSTOM_OBJECT_CLASSIFICATION_VGG16/TRAIN',    # directory train images
        x_col = "Filenames",                       # name of the column which contains the filenames of the images
        y_col = "labels",                          # name of the column which contains the class name
        batch_size = 32 * num_gpus,                # size of the batch
        seed = 42,                                 # set seed for reproducible results
        shuffle = True,                            # shuffle images
        class_mode = "categorical",                # class mode
        #classes = ["sunflower"],                  # class name
        target_size = (224, 224),                  # size of image
        )

# Flow validation images from validation dataframe in batches using valid_datagen generator
valid_generator = valid_datagen.flow_from_dataframe(

        dataframe = df_test,
        directory = '/pasquale/CUSTOM_OBJECT_CLASSIFICATION_VGG16/VALIDATION', # directory validation images
        x_col = "Filenames",
        y_col = "labels",
        batch_size = 32 * num_gpus,
        seed = 42,
        shuffle = True,      
        class_mode = "categorical",
        #classes = ["sunflower"],
        target_size = (224, 224)
        )

# Flow test images from test dataframe in batches using test_datagen generator
test_generator = test_datagen.flow_from_dataframe(
    
        dataframe = df_test,
        directory = '/pasquale/CUSTOM_OBJECT_CLASSIFICATION_VGG16/TEST',   # directory test images
        x_col = "Filenames",
        y_col = None,
        batch_size = 32 * num_gpus,
        seed = 42,
        shuffle = False,     # PAY ATTENTION: SHUFFLE = FALSE in the test data, because
                                        # we need to yield the images in “order”, 
                                        # to predict the outputs and match them with their unique filenames
        class_mode = None,
        target_size = (224,224)
         )

Then, we can define the size of the steps for training, validation and test.     
The step is defined as the ratio between the number of images (in the specific set) and the batch size (in the specific set).

In [ ]:
# Defining STEP SIZE 
STEP_SIZE_TRAIN = train_generator.n/train_generator.batch_size     # step size training
STEP_SIZE_VALID = valid_generator.n/valid_generator.batch_size     # step size validation
STEP_SIZE_TEST = test_generator.n/test_generator.batch_size        # step size test

## Model Building (Transfer Learning) and Training

In order to build our Custom Classifier we will use the VGG16 architecture, that is a network pre-trained on a large dataset (ImageNet dataset).    
Such a network would have already learned features that are useful for most computer vision problems, and leveraging such features would allow us to reach a better performance than any method that would only rely on the available data.

Let's first have a look at VGG16 architecture:

![title](VGG16.png) 

In order to build our Custom Model, we take only the Convolutional Layers from this architecture and we add our custom Fully Connected Layers on top of Convolutional part of VGG16 architecture.     
Then, we freeze the entire Convolutional block (that works as features extractor) and we make predictions by training only the Fully Connected Layers on our own dataset.   
The Last Layer of our Custom Classifier contains a single node (1 class) with sigmoid activation function to output probability of sunflower for each specific example (image).   
The technique described above is known as "Transfer Learning", applied to Image Classification task. 

As Keras documentation well explains:   

"Transfer Learning consists of taking features learned on one problem, and leveraging them on a new, similar problem. Transfer Learning is usually done for tasks where your dataset has too little data to train a full-scale model from scratch.   
The most common incarnation of Transfer Learning in the context of deep learning is the following workflow:

    - Take layers from a previously trained model.
    - Freeze them, so as to avoid destroying any of the information they contain during future training rounds.
    - Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into
      predictions on a new dataset.
    - Train the new layers on your dataset."


In [ ]:
# MirroredStrategy() is used for synchronous distributed training on multiple GPUs
mirrored_strategy = tf.distribute.MirroredStrategy()        # all GPUs involved


with mirrored_strategy.scope():

    ## VGG16 MODEL LOADING ##
    #########################
    
    vgg16_model = VGG16(
                     include_top = False,              # Leave out the 3 fully connected layers
                     weights = 'imagenet'              # Pre-trained weights on Imagenet dataset
                     )
    
    ## FREEZING CONVOLUTIONAL BLOCKS ##
    ###################################
    
    for layer in vgg16_model.layers:                   # make NON-TRAINABLE LAYERS
        layer.trainable = False
        
    
    ## CREATE CLASSIFICATION HEAD ON TOP OF VGG16 MODEL ##
    ######################################################
    
    # Create Input Layer
    input_layer = Input(shape=(224, 224, 3), name = 'input')

    # Generate output from VGG16 convolutional block 
    output_vgg16_model = vgg16_model(input_layer)

    # Add the Fully-Connected Layers 
    x = Flatten(name = 'flatten')(output_vgg16_model)                  # Flatten
    x = Dense(4096, activation='relu', name = 'fc1')(x)                # 1st FCLayer
    x = layers.Dropout(0.4)(x)                                         # Dropout
    x = Dense(4096, activation='relu', name = 'fc2')(x)                # 2nd FCLayer
    x = layers.Dropout(0.4)(x)                                         # Dropout
    x = Dense(1, activation = 'sigmoid', name = 'predictions')(x)      # Output Layer

    #Create Custom Model  
    final_vgg16_model = Model(inputs = input_layer, outputs = x)
    
    
    ## PARAMETERS AND METRICS SETTING ##
    ####################################
    
    # metrics
    precision = tf.keras.metrics.Precision(name = "precision")                               # precision
    recall = tf.keras.metrics.Recall(name = "recall")                                        # recall
    f1_score = tfa.metrics.F1Score(num_classes =  1, threshold = 0.5, name = 'f1_score')     # f1 score
    auc = tf.keras.metrics.AUC(name = "auc")                                                 # AUC
    tp = tf.keras.metrics.TruePositives(name='tp')                                           # True Positives
    fp = tf.keras.metrics.FalsePositives(name='fp')                                          # False Positives
    tn = tf.keras.metrics.TrueNegatives(name='tn')                                           # True Negatives
    fn = tf.keras.metrics.FalseNegatives(name='fn')                                          # False Negatives
    
    
    # Define EARLY STOPPING  (we monitor validation f1 score)
    early_stopping = EarlyStopping(
                            monitor = "val_f1_score",       # metric to be monitored
                            mode = "max",                   # stop when the quantity monitored has stopped increasing
                            restore_best_weights = True,    # restore weights from the epoch with the best value 
                                                            # of monitored quantity
                            patience = 50                   # wait 50 epochs without improving validation f1_score
                            )                  
    
    
    # Define MODEL CHECKPOINT
    # ModelCheckpoint allows us to save the best model with the best weights during the training.
    # The best model observed during training is defined by a chosen performance metric on the validation dataset.
    mc = ModelCheckpoint(
                          'sunflower_classifier_best.h5',  # model saving name
                           monitor = 'val_f1_score',       # metric to be monitored
                           mode = 'max',                   # increasing value of the metric
                           verbose = 1,                    # verbose mode
                           save_best_only = True           # save only the model that has achieved the best performance
                         )
    
    
    # Adam Optimizer
    optimizer = tf.keras.optimizers.Adam(
                           learning_rate=0.001 * num_gpus)

    
    ## COMPILE THE MODEL ##
    #######################
    
    final_vgg16_model.compile( 
                       optimizer = optimizer,                                                     # optimizer
                       loss = 'binary_crossentropy',                                              # loss function
                       metrics = ["accuracy", precision, recall, f1_score, auc, fp, tp, fn, tn]   # metrics
                        )
    
    
    
    
    ## MODEL TRAINING ##
    ####################

    history_vgg16 = final_vgg16_model.fit(
                            
                            train_generator,                    # train generator with images and labels
                            steps_per_epoch = STEP_SIZE_TRAIN,  # total number of steps (batches of samples)... 
                                                                # ...before declaring one epoch finished
                            validation_data = valid_generator,  # validation generator with images and labels
                            validation_steps = STEP_SIZE_VALID, # validation step
                            epochs = 500,                       # number of training epochs
                            callbacks = [early_stopping, mc]    # callbacks
                            )
    

## Plot Training and Validation Metrics
### (Model Performance)

After training the Classifier, we can have a first look at the perfomance of our model by comparing training and validation metrics (loss, precision, recall, f1 score).     
For istance, a low training loss and a high validation loss can be interpreted as a sign of overfitting.

In [ ]:
# LOSS
plt.plot(history_vgg16.history['loss'], label='train loss')
plt.plot(history_vgg16.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# RECALL
plt.plot(history_vgg16.history['recall'], label='train recall')
plt.plot(history_vgg16.history['val_recall'], label='validation recall')
plt.legend()
plt.show()

# PRECISION
plt.plot(history_vgg16.history['precision'], label='train precision')
plt.plot(history_vgg16.history['val_precision'], label='validation precision')
plt.legend()
plt.show()

# F1-SCORE 
plt.plot(history_vgg16.history['f1_score'], label='train f1_score')
plt.plot(history_vgg16.history['val_f1_score'], label='validation f1_score')
plt.legend()
plt.show()

## Predictions on test images

Now, let's analize in detail how our Custom Classifier performs on test data by making predictions on brand new images...

In [ ]:
test_generator.reset()       # reset test generator

threshold = 0.5              # threshold to distinguish between positive (1) and negative class (0)

# predict probabilities for test images
pred_prob = final_vgg16_model.predict(
                            
                            test_generator,               # images in test generator
                            steps = STEP_SIZE_TEST,       # step size
                            verbose=1                     # verbose
                            )

# predict class for test images
y_pred = (pred_prob > threshold).astype(int)              # convert probabilities into predicted classes


# extract ground truth labels from test images
y_true = []
for i in df_test.labels:
    if "sunflower" in i:
        y_true.append(1)
    else:
        y_true.append(0)

## Evaluate Classifier Performance

We can evaluate in detail the performance of our Custom Classifier by having a look at the Confusion Matrix, which allows us to clearly compare actual and predicted labels. Indeed, we can appreciate True Positives, True Negatives, False Positives and False Negatives values.   
Furthermore, Classification Report also helps us in evaluating Precision, Recall and F1 score of our model in both images with sunflowers and images without sunflowers.    
All metrics will guide our decisions to fine-tune the parameters of our model in order to achieve the best performance.

In [ ]:
## CONFUSION MATRIX ##
######################

SUNFLOWER_CONFUSION_MATRIX = pd.DataFrame(
                 
                 confusion_matrix(y_true, y_pred), 
                 columns = ["NO_SUNFLOWER_pred","SUNFLOWER_pred"], 
                 index = ["NO_SUNFLOWER_true", "SUNFLOWER_true"]
                  )

TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()



## CLASSIFICATION REPORT ##
###########################

print("######   CLASSIFICATION REPORT (SUNFLOWER)   ######\n###################################################\n")
print(classification_report(y_true, y_pred, 
                            target_names = ["NO SUNFLOWER IMAGES", "SUNFLOWER IMAGES"]))

print("CONFUSION MATRIX VALUES", "TN:",TN, "FP:", FP, "FN:", FN, "TP:", TP)

SUNFLOWER_CONFUSION_MATRIX

## Plot Precision-Recall Curve 

We can also evaluate the performance of our model by plotting the Precision-Recall Curve. Precision-Recall Curve is typically used in binary classification to study the output of a classifier.   
The Precision-Recall Curve shows the trade-off between precision and recall for different threshold.    
A high area under the curve represents both high recall and high precision. High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all positive results (high recall).  A perfect skill classifier has full precision and recall with a dot in the top-right corner.   
If we are interested in a threshold which results in the best balance of precision and recall, then this is the same as finding the threshold that optimize the F1 score.      
So, we can also plot the point corresponding to the OPTIMAL THRESHOLD (MAX F1 SCORE) on the precision-recall curve.    
The idea is that, once found this optimal threshold, it could then be used when making probability predictions in the future that must be converted from probabilities to class labels.

In [ ]:
## PRECISION-RECALL CURVE AND OPTIMAL THRESHOLD ##
##################################################

precision, recall, thresholds = precision_recall_curve(y_true, pred_prob)      # precision-recall curve

# calculate F1 score
f1 = f1_score(y_true, y_pred)

# calculate precision-recall AUC
auc_score = auc(recall, precision)

# summarize scores
print("\nF1 SCORE:", f1, "  AUC SCORE:" ,auc_score)


fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = argmax(fscore)
print('\nBest Threshold=%f, F1-Score=%.3f' % (thresholds[ix], fscore[ix]))

# plot the precision-recall curves
no_skill = len([ i for i in y_true if i == 1]) / len(y_true)                   # no-skill model
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(recall, precision, marker='.', label='Model')
plt.scatter(recall[ix], precision[ix], marker='o', color='black', label='MAX F1-SCORE')  # point max f1-score
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()

## Prediction on single image

We can also try to test our Custom Classifier on a single image...

In [ ]:
path = "/pasquale/CUSTOM_OBJECT_CLASSIFICATION_VGG16/"             # image loading path

image_filename = "sunflower_img_10.jpg"                                      # filename


## IMAGE LOADING AND PREPROCESSING ##
#####################################

img = load_img(
               path + image_filename,             # image filename
               target_size=(224,224)              # load image with a specific size 
               ) 

img = img_to_array(img)                           # convert the pixels to a numpy array

height, width, channels = img.shape               # retrieve height, width and channels of the image

img = img.reshape((1, height, width, channels))   # reshape the image to add an extra dimension
                                                  # (extra dimension is number of samples)

img = preprocess_input(img)                       # preprocess the image to use as input into the network


## PREDICTION ##
################

prediction_prob = final_vgg16_model.predict(img)        # model prediction on single image

print(image_filename, " PROBABILITY OF SUNFLOWER IN THE IMAGE:", prediction_prob[0][0])

...finally, don't forget to clear GPU memory

In [ ]:
## CLEAR GPU MEMORY ##
######################

K.clear_session()
cuda.select_device(0)            # select specific GPU device 
cuda.close()